In [1]:
%matplotlib inline
import numpy as np
from numpy import random
import argparse
import subprocess
import os
from scipy.spatial import cKDTree
from netCDF4 import Dataset, default_fillvals
import xarray as xr
import xarray.ufuncs as xu


create 5 x 5 array of random integers

In [79]:
# five_by_five = random.random_integers(0, 100, (5, 5))
five_by_five = np.array(range(1, 26, 1)).reshape(5,5)

In [82]:
five_by_five

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

lats and lons for 5x5 array and 10x10 array

In [45]:
lats_5x5 = np.arange(41, 43.5, 0.5)
lons_5x5 = np.arange(93, 95.5, 0.5)

lats_10x10 = np.arange(40.75, 43.25, 0.25)
lons_10x10 = np.arange(92.75, 95.25, 0.25)

In [58]:
xv_5x5, yv_5x5 = np.meshgrid(lats_5x5, lons_5x5)
xv_10x10, yv_10x10 = np.meshgrid(lats_10x10, lons_10x10)

In [46]:
def lon_lat_to_cartesian(lon, lat, radius=1):
    """
    calculates lon, lat coordinates of a point on a sphere with
    radius radius
    """

#     # Unpack xarray object into plane arrays
#     if hasattr(lon, 'data'):
#         lon = lon.data
#     if hasattr(lat, 'data'):
#         lat = lat.data

#     if lon.ndim != lat.ndim:
#         raise ValueError('coordinate must share the same number of dimensions')

#     if lon.ndim == 1:
#         lon, lat = np.meshgrid(lon, lat)

    lon_r = xu.radians(lon)
    lat_r = xu.radians(lat)

    x = radius * xu.cos(lat_r) * xu.cos(lon_r)
    y = radius * xu.cos(lat_r) * xu.sin(lon_r)
    z = radius * xu.sin(lat_r)

    return x.flatten(), y.flatten(), z.flatten()

In [47]:
def latlon2yx(plons, plats, glons, glats):
    '''find y x coordinates'''

    if glons.ndim == 1 or glats.ndim == 1:
        glons, glats = np.meshgrid(glons, glats)

    px, py, pz = lon_lat_to_cartesian(plons, plats)
    gx, gy, gz = lon_lat_to_cartesian(glons, glats)

    combined = np.dstack(([gx, gy, gz]))[0]
    points = list(np.vstack((px, py, pz)).transpose())

    mytree = cKDTree(combined)
    dist, indexes = mytree.query(points, k=1)
    y, x = np.unravel_index(np.array(indexes), glons.shape)
    return y, x

code used for RASM parameters and domain file array values 

In [48]:
'''# 25km land lats/lngs
yc_land_25km = domain_25km.yc.where(domain_25km.mask == 1, drop=True)
xc_land_25km = domain_25km.xc.where(domain_25km.mask == 1, drop=True)
land_lats_25km = yc_land_25km.values[~np.isnan(yc_land_25km.values)]
land_lons_25km = xc_land_25km.values[~np.isnan(xc_land_25km.values)]
nj_inds_25km, ni_inds_25km = np.nonzero(domain_25km.mask.values)

# 50km land lats/lngs
yc_land_50km = params_50km.yc.where(params_50km.run_cell == 1, drop=True)
xc_land_50km = params_50km.xc.where(params_50km.run_cell == 1, drop=True)
land_lats_50km = yc_land_50km.values[~np.isnan(yc_land_50km.values)]
land_lons_50km = xc_land_50km.values[~np.isnan(xc_land_50km.values)]
nj_inds_50km, ni_inds_50km = np.nonzero(params_50km.run_cell.values)'''

'# 25km land lats/lngs\nyc_land_25km = domain_25km.yc.where(domain_25km.mask == 1, drop=True)\nxc_land_25km = domain_25km.xc.where(domain_25km.mask == 1, drop=True)\nland_lats_25km = yc_land_25km.values[~np.isnan(yc_land_25km.values)]\nland_lons_25km = xc_land_25km.values[~np.isnan(xc_land_25km.values)]\nnj_inds_25km, ni_inds_25km = np.nonzero(domain_25km.mask.values)\n\n# 50km land lats/lngs\nyc_land_50km = params_50km.yc.where(params_50km.run_cell == 1, drop=True)\nxc_land_50km = params_50km.xc.where(params_50km.run_cell == 1, drop=True)\nland_lats_50km = yc_land_50km.values[~np.isnan(yc_land_50km.values)]\nland_lons_50km = xc_land_50km.values[~np.isnan(xc_land_50km.values)]\nnj_inds_50km, ni_inds_50km = np.nonzero(params_50km.run_cell.values)'

get indices for mapping back

## NOTE: plats/plons will come from the coarser grid, the glons/glats will be from the higher res grid 

In [63]:
y_inds, x_inds = latlon2yx(yv_10x10, xv_10x10, yv_5x5, xv_5x5)

map coarser res values into higher res array 

In [73]:
tenbyten = np.ones(shape=(100, 1))
for i in range(len(y_inds)):
    tenbyten[i] = five_by_five[y_inds[i], x_inds[i]]
tenbyten = tenbyten.reshape(10, 10)


check values of 5x5 array

In [74]:
five_by_five

array([[ 8, 84, 38, 61, 76],
       [39, 82, 43, 66,  2],
       [95, 69, 25, 17, 34],
       [85, 45, 39, 43, 78],
       [19, 34, 31, 86, 59]])

look at whether or not 10x10 array makes sense given the 5x5 original one

In [75]:
tenbyten

array([[  8.,   8.,  84.,  84.,  38.,  38.,  61.,  61.,  76.,  76.],
       [  8.,   8.,   8.,  84.,  84.,  38.,  38.,  61.,  61.,  76.],
       [ 39.,  39.,  82.,  82.,  43.,  43.,  66.,  66.,   2.,   2.],
       [ 39.,  39.,  82.,  82.,  82.,  43.,  43.,  66.,  66.,   2.],
       [ 39.,  39.,  82.,  82.,  43.,  43.,  66.,  66.,   2.,   2.],
       [ 95.,  95.,  95.,  69.,  69.,  25.,  25.,  17.,  17.,  34.],
       [ 85.,  95.,  45.,  69.,  25.,  25.,  43.,  17.,  34.,  34.],
       [ 85.,  85.,  45.,  45.,  45.,  39.,  39.,  43.,  43.,  78.],
       [ 85.,  85.,  45.,  45.,  39.,  39.,  43.,  43.,  78.,  78.],
       [ 19.,  19.,  19.,  34.,  34.,  31.,  31.,  86.,  86.,  59.]])

try using `scipy.misc.imresize` instead 

In [ ]:
five_by_five = np.array(range(1, 26, 1)).reshape(5,5)

In [84]:
from scipy import misc
ten_by_ten = misc.imresize(five_by_five, size=(10, 10), interp='nearest', mode=None)

In [87]:
five_by_five

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

In [85]:
ten_by_ten

array([[  0,   0,  11,  11,  21,  21,  32,  32,  42,  42],
       [  0,   0,  11,  11,  21,  21,  32,  32,  42,  42],
       [ 53,  53,  64,  64,  74,  74,  85,  85,  96,  96],
       [ 53,  53,  64,  64,  74,  74,  85,  85,  96,  96],
       [106, 106, 117, 117, 127, 127, 138, 138, 149, 149],
       [106, 106, 117, 117, 127, 127, 138, 138, 149, 149],
       [159, 159, 170, 170, 181, 181, 191, 191, 202, 202],
       [159, 159, 170, 170, 181, 181, 191, 191, 202, 202],
       [212, 212, 223, 223, 234, 234, 244, 244, 255, 255],
       [212, 212, 223, 223, 234, 234, 244, 244, 255, 255]], dtype=uint8)

try using `scipy.ndimage.zoom` instead

In [88]:
from scipy import ndimage
ten_by_ten = ndimage.zoom(five_by_five, 2, order=0)

In [89]:
ten_by_ten

array([[ 1,  1,  2,  2,  3,  3,  4,  4,  5,  5],
       [ 1,  1,  2,  2,  3,  3,  4,  4,  5,  5],
       [ 6,  6,  7,  7,  8,  8,  9,  9, 10, 10],
       [ 6,  6,  7,  7,  8,  8,  9,  9, 10, 10],
       [11, 11, 12, 12, 13, 13, 14, 14, 15, 15],
       [11, 11, 12, 12, 13, 13, 14, 14, 15, 15],
       [16, 16, 17, 17, 18, 18, 19, 19, 20, 20],
       [16, 16, 17, 17, 18, 18, 19, 19, 20, 20],
       [21, 21, 22, 22, 23, 23, 24, 24, 25, 25],
       [21, 21, 22, 22, 23, 23, 24, 24, 25, 25]])